In [3]:
import pandas as pd

In [8]:
from turnosenfermeria import TurnosEnfermeria

In [9]:
tamano_individuo = 21

In [5]:
preferencias = pd.read_csv("./data/datos_enfermeria.txt", sep=' ')
print (preferencias)

   Nombre  Pref_M  Pref_T  Pref_N
0   Enf_A       1       1       0
1   Enf_B       1       0       1
2   Enf_C       1       1       1
3   Enf_D       0       1       0
4   Enf_E       0       0       1
5   Enf_F       0       1       1
6   Enf_G       1       1       0
7   Enf_H       0       0       1
8   Enf_I       1       1       1
9   Enf_J       1       1       0
10  Enf_K       1       1       1
11  Enf_L       1       1       0
12  Enf_M       1       0       1


In [6]:
enfermeras = preferencias["Nombre"]
print (enfermeras)

0     Enf_A
1     Enf_B
2     Enf_C
3     Enf_D
4     Enf_E
5     Enf_F
6     Enf_G
7     Enf_H
8     Enf_I
9     Enf_J
10    Enf_K
11    Enf_L
12    Enf_M
Name: Nombre, dtype: object


In [7]:
turnos = TurnosEnfermeria (enfermeras, preferencias)

In [14]:
def coste_individuo (individuo, poblacion):
    
    if len(individuo) != tamano_individuo:
         raise ValueError("el tamaño del individuo debe ser igual a: ", tamano_individuo)

    infraccionesTurnosConsecutivos = InfraccionesTurnosConsecutivos(individuo)
    infraccionesTurnosMismoDia = InfraccionesTurnosMismoDia(individuo)
    infraccionesTurnosPorSemana = InfraccionesTurnosPorSemana(individuo)[1]
    infraccionesEnfermerPorTurno = InfraccionesEnfermerPorTurno(individuo)[1]
    infraccionesPreferenciaTurnos = InfraccionesPreferenciaTurnos(individuo)[1]
    
    
    infraccionesRestriccionesHard = infraccionesTurnosConsecutivos + infraccionesTurnosMismoDia + infraccionesEnfermerPorTurno + infraccionesTurnosPorSemana
    infraccionesRestriccionesSoft = infraccionesPreferenciaTurnos

    return turnos.factorPenalizacionRestriccionDura * infraccionesRestriccionesHard + infraccionesRestriccionesSoft,

    

In [46]:
def InfraccionesTurnosConsecutivos (individuo):
    infracciones = 0

    for turno1, turno2 in zip (individuo, individuo[1:]):
        if turno1 == 1 and turno2 == 1:
            infracciones +=1

    return infracciones

    
    

In [44]:
def InfraccionesTurnosMismoDia (individuo):
    infracciones = 0
    indiceTurno = 0

    for i in range((7 * turnos.semanas)):
        turno=individuo[indiceTurno:indiceTurno+turnos.turnosPorDia]
        print (turno)

        if turno[0]==1 & turno[2]==1:
            infracciones +=1
        indiceTurno+=turnos.turnosPorDia

    return infracciones
        
        

In [57]:
def InfraccionesTurnosPorSemana (individuo):
    infracciones = 0
    lista_TurnosSemanales = []

    for i in range(0, turnos.semanas * turnos.turnosPorSemana, turnos.turnosPorSemana):
        turnosSemanales = sum(individuo[i:i + turnos.turnosPorSemana])
        lista_TurnosSemanales.append(turnosSemanales)
        if turnosSemanales > turnos.maxTurnosSemana:
            infracciones += turnosSemanales - turnos.maxTurnosSemana

    return lista_TurnosSemanales, infracciones

In [61]:
def InfraccionesEnfermerPorTurno(poblacion):
    listaTotalPorTurno = [sum(turno) for turno in zip(*poblacion)]

    infracciones = 0
        
    for indiceTurno, numDeEnfermer in enumerate(listaTotalPorTurno):
        indiceTurnoDiario = indiceTurno % turnos.turnosPorDia  # -> 0, 1, o 2 para los tres turnos al dia
        if (numDeEnfermer > turnos.maxTurnosDia[indiceTurnoDiario]):
            infracciones += numDeEnfermer - turnos.maxTurnosDia[indiceTurnoDiario]
        elif (numDeEnfermer < self.minTurnosDia[indiceTurnoDiario]):
            infracciones += self.minTurnosDia[indiceTurnoDiario] - numDeEnfermer

    return listaTotalPorTurno, infracciones
    

In [ ]:
def countInfraccionesEnfermerPorTurno(self, turnosEnfermDict):
        """
        Cuenta el numero de infracciones debidas al numero de enfermer@s por turno
        :param turnosEnfermDict: diccionario con el turno específico de un enfermer@
        :return: numero de infracciones encontradas
        """
        # sumamos los turnos de todas las enfermeras:
        listaTotalPorTurno = [sum(turno) for turno in zip(*turnosEnfermDict.values())]

        infracciones = 0
        # iteramos en los turnos y obtenemos las infracciones:
        for indiceTurno, numDeEnfermer in enumerate(listaTotalPorTurno):
            indiceTurnoDiario = indiceTurno % self.turnosPorDia  # -> 0, 1, o 2 para los tres turnos al dia
            if (numDeEnfermer > self.maxTurnosDia[indiceTurnoDiario]):
                infracciones += numDeEnfermer - self.maxTurnosDia[indiceTurnoDiario]
            elif (numDeEnfermer < self.minTurnosDia[indiceTurnoDiario]):
                infracciones += self.minTurnosDia[indiceTurnoDiario] - numDeEnfermer

        return listaTotalPorTurno, infracciones